# **Clarifai as destination connector using Unstructured.IO**

This Notebook walk you through the step by step guide on how to utilise Clarifai as your destination connector using Unstructured.IO and import raw files from various source connectors. For the demo we will be using S3 bucket as our source connector which has out raw text files.

### Setup
Install necessary libraries.

In [ ]:
! pip install "unstructured[clarifai]" #make sure the unstructured version is 0.13 or above

! pip install "unstructured[s3]" #since our source is S3

## S3 setup

In [ ]:
access_key='YOUR_S3_ACCESS_KEYS'
secret_access='YOUR_S3_SECRET_ACCESS_KEYS'

## Clarifai and S3 imports

In [ ]:
import os
from unstructured.ingest.connector.fsspec.s3 import S3AccessConfig, SimpleS3Config
from unstructured.ingest.interfaces import (
    PartitionConfig,
    ProcessorConfig,
    ChunkingConfig,
    ReadConfig,
)
from unstructured.ingest.runner import S3Runner

from unstructured.ingest.connector.clarifai import (
    ClarifaiAccessConfig,
    ClarifaiWriteConfig,
    SimpleClarifaiConfig,
)

from unstructured.ingest.runner.writers.base_writer import Writer
from unstructured.ingest.runner.writers.clarifai import (
    ClarifaiWriter,
)

## Clarifai_writer func which will configure the target clarifai app where the ingested documents will be loaded with initialising clarifai PAT as `api_key`

In [ ]:
def clarifai_writer() -> Writer:
    return ClarifaiWriter(
        connector_config=SimpleClarifaiConfig(
            access_config=ClarifaiAccessConfig(
                api_key= "YOUR_PAT"
            ),
            app_id= "YOUR_CLARIFAI_APP",
            user_id= "USER_ID"
            ),
        write_config=ClarifaiWriteConfig()
    )

## Let's now package the writer and runner together, make sure you have your S3 bucket URI ready.

In [ ]:
if __name__ == "__main__":
    writer = clarifai_writer()
    runner = S3Runner(
        processor_config=ProcessorConfig(
            verbose=True,
            output_dir="s3-output-local",
            num_processes=2,
        ),
        read_config=ReadConfig(),
        partition_config=PartitionConfig(),
        connector_config=SimpleS3Config(
            access_config=S3AccessConfig(
                key=access_key,
                secret=secret_access,
            ),
            remote_url="s3://my-trial-bucket/procurement.txt",
        ),
        writer=writer,
        writer_kwargs={},
    )
    runner.run()